In [43]:
import pandas as pd
import duckdb

### Data

In [65]:
db = duckdb.connect('./data/data.db')
base_df = db.sql(
"""
SELECT
    * Exclude(Home_Score, Away_Score)
FROM game_features
"""
).df()
rank_df = db.sql(
f"""
SELECT
    Year, Week, Team,
    ROW_NUMBER() OVER (PARTITION BY Year, Week ORDER BY Rating DESC) AS Rank
FROM nfl_rankings
"""
).df()
hrank_df = db.sql(
f"""
SELECT
    gf.* EXCLUDE(Spread, Home_Rank, Away_Rank, Home_Won),
    gf.Week - r_ht.Week as Rank_Age,
    r_ht.Rank as Home_Rank,
    r_at.Rank as Away_Rank,

    gf.Home_Won
FROM base_df as gf
INNER JOIN rank_df as r_ht
    ON r_ht.Year = gf.Year AND r_ht.Team = gf.Home_Team AND r_ht.Week <= gf.Week
INNER JOIN rank_df as r_at
    ON r_at.Year = gf.Year AND r_at.Team = gf.Away_Team AND r_at.Week <= gf.Week
WHERE 
    r_ht.Week = r_at.Week
    -- AND gf.Year >= 2020
ORDER BY gf.Year, gf.Week, gf.Home_Team, gf.Away_Team, r_ht.Week ASC
"""
).df()
db.close()

In [63]:
list(base_df.columns)


['Year',
 'Week',
 'Date',
 'Home_Team',
 'Away_Team',
 'Is_Neutral',
 'Spread',
 'Home_Rank',
 'Away_Rank',
 'Home_Days_Since_Last_Game',
 'Away_Days_Since_Last_Game',
 'Home_Games_Played',
 'Away_Games_Played',
 'Home_Wins',
 'Away_Wins',
 'Home_Losses',
 'Away_Losses',
 'Home_Won']

### Viz

In [83]:
import plotly.express as px
import numpy as np
import pandas as pd

# Bucketize the ranks
df = base_df.copy()
df['Home_Rank_Bucket'] = pd.cut(df['Home_Rank'], bins=np.arange(1, 36, 4), right=False, labels=[f'R:{i}-{i+3}' for i in range(1, 32, 4)])
df['Away_Rank_Bucket'] = pd.cut(df['Away_Rank'], bins=np.arange(1, 36, 4), right=False, labels=[f'R:{i}-{i+3}' for i in range(1, 32, 4)])

# Reworked to be a heatmap showing home win probability
fig = px.density_heatmap(
    df.dropna(subset=['Home_Rank_Bucket', 'Away_Rank_Bucket']),
    x='Home_Rank_Bucket',
    y='Away_Rank_Bucket',
    z='Home_Won',
    text_auto=".0%", # Format text in cells as percentages
    histfunc='avg',
    title='Home Win Probability: Home Rank vs. Away Rank (Bucketed)',
    labels={'color': 'Home Win Probability'},
    color_continuous_scale='RdYlGn', # Red-Yellow-Green scale
    template='plotly_dark',
    category_orders={'Home_Rank_Bucket': [f'R:{i}-{i+3}' for i in range(1, 32, 4)], 'Away_Rank_Bucket': [f'R:{i}-{i+3}' for i in range(1, 32, 4)]}
)

fig.update_layout(
    xaxis_title="Home Team Rank (Bucketed)",
    yaxis_title="Away Team Rank (Bucketed)",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()


  - The diag still has prob to the side of the home team.
  - As we go from higher ranking teams in the diag to lower, we see things get more random

In [84]:
import plotly.express as px
import pandas as pd
import numpy as np

# 1. Calculate the rank difference.
# A negative value means the home team has a better (lower) rank.
df = base_df.copy()
df['Rank_Diff'] = df['Home_Rank'] - df['Away_Rank']

# 2. Group by the rank difference and calculate home win probability.
# We sort by 'Rank_Diff' to ensure the x-axis is ordered correctly.
win_prob_by_rank_diff = (
    df.groupby('Rank_Diff')['Home_Won']
    .mean()
    .reset_index(name='Home_Win_Probability')
    .sort_values('Rank_Diff')
)

# 3. Create the bar plot.
fig = px.bar(
    win_prob_by_rank_diff,
    x='Rank_Diff',
    y='Home_Win_Probability',
    title='Home Win Probability vs. Rank Difference',
    labels={
        'Rank_Diff': 'Rank Difference (Home Rank - Away Rank)',
        'Home_Win_Probability': 'Home Win Probability'
    },
    template='plotly_dark'
)

# 4. Update the layout for better readability and aesthetics.
fig.update_layout(
    xaxis_title="Rank Difference (Home Team - Away Team)",
    yaxis_title="Home Win Probability",
    yaxis=dict(
        tickformat=".0%",  # Format y-axis labels as percentages
        range=[0, 1]       # Set y-axis range from 0 to 1
    ),
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()


In [85]:
import plotly.express as px
import pandas as pd
import numpy as np

# 1. Calculate the rank difference.
# A negative value means the home team has a better (lower) rank.
df = base_df.copy()
df['Rank_Diff'] = df['Home_Rank'] - df['Away_Rank']

# 2. Bucketize the rank difference into intervals of size 4.
# We define the edges of the bins from -32 to +32 with a step of 4.
max_abs_diff = df['Rank_Diff'].abs().max()
# Ensure bins cover the full range of data, stepping by 4
bin_edge = int(np.ceil(max_abs_diff / 4) * 4)
bins = np.arange(-bin_edge, bin_edge + 1, 4)

# Create meaningful labels for each bucket.
labels = [f'{i} to {i + 3}' for i in bins[:-1]]

df['Rank_Diff_Bucket'] = pd.cut(
    df['Rank_Diff'],
    bins=bins,
    labels=labels,
    right=False, # Intervals are [left, right)
    include_lowest=True
)

# 3. Group by the rank difference bucket and calculate home win probability.
# We use observed=False to include all possible buckets in the output, even if empty.
win_prob_by_bucket = (
    df.groupby('Rank_Diff_Bucket', observed=False)['Home_Won']
    .mean()
    .reset_index(name='Home_Win_Probability')
)

# 4. Create the bar plot.
fig = px.bar(
    win_prob_by_bucket,
    x='Rank_Diff_Bucket',
    y='Home_Win_Probability',
    title='Home Win Probability vs. Bucketed Rank Difference',
    labels={
        'Rank_Diff_Bucket': 'Rank Difference Bucket (Home Rank - Away Rank)',
        'Home_Win_Probability': 'Home Win Probability'
    },
    template='plotly_dark'
)

# 5. Update the layout for better readability and aesthetics.
fig.update_layout(
    xaxis_title="Rank Difference Bucket (Home Team - Away Team)",
    yaxis_title="Home Win Probability",
    yaxis=dict(
        tickformat=".0%",  # Format y-axis labels as percentages
        range=[0, 1]       # Set y-axis range from 0 to 1
    ),
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()


In [86]:
import pandas as pd
import numpy as np
import plotly.express as px


df = hrank_df.copy()

# 1. NEW: Calculate the overall home win percentage for the benchmark
# This is the average of the Home_Won column, converted to a percentage.
home_win_benchmark = (df['Home_Won'].mean()) * 100
print(f"Overall Home Team Win % (Benchmark): {home_win_benchmark:.2f}%")


# 2. Determine if the prediction based on rank was correct
def is_prediction_correct(row):
    if row['Home_Rank'] < row['Away_Rank']:
        return row['Home_Won'] == 1
    elif row['Away_Rank'] < row['Home_Rank']:
        return row['Home_Won'] == 0
    else:
        return None

df['Prediction_Correct'] = df.apply(is_prediction_correct, axis=1)

# 3. Calculate the prediction accuracy for each Rank_Age
accuracy_by_age = df.groupby('Rank_Age')['Prediction_Correct'].mean().reset_index()
accuracy_by_age.rename(columns={'Prediction_Correct': 'Prediction_Accuracy'}, inplace=True)
accuracy_by_age['Prediction_Accuracy'] = accuracy_by_age['Prediction_Accuracy'] * 100

# 4. Create the visualization using Plotly
fig = px.line(
    accuracy_by_age,
    x='Rank_Age',
    y='Prediction_Accuracy',
    title='Predictive Power of Team Rank vs. Age of Rank',
    labels={
        'Rank_Age': 'Age of Rank (in weeks)',
        'Prediction_Accuracy': 'Prediction Accuracy (%)'
    },
    markers=True,
    template='plotly_dark'
)

# 5. NEW: Add the horizontal benchmark line to the figure
fig.add_hline(
    y=home_win_benchmark, 
    line_dash="dot",  # Style of the line (e.g., 'dot', 'dash')
    annotation_text=f"Home Win % Benchmark ({home_win_benchmark:.1f}%)", 
    annotation_position="bottom right",
    annotation_font_color="yellow"
)


# 6. Update the layout as before
fig.update_layout(
    xaxis_title='Age of Rank (0 = Most Current)',
    yaxis_title='Accuracy of Rank in Predicting Winner (%)',
    yaxis=dict(
        tickformat=".1f",
    ),
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()

Overall Home Team Win % (Benchmark): 55.63%


In [87]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty=None)
lr.fit(pd.DataFrame(
    hrank_df["Home_Rank"] - hrank_df["Away_Rank"]
), hrank_df[["Home_Won"]])

/Users/anzekravanja/Projects/smart-nfl-survivor-v2/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



,penalty,None
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [88]:
lr.predict_proba([
    [-1],
    [-2],
    [-20],
    [9]
])[:, 1]

array([0.5732087 , 0.58543131, 0.77693761, 0.44853296])

In [89]:
import pandas as pd
import numpy as np
import plotly.express as px


df = hrank_df.copy()

home_win_benchmark = (df['Home_Won'].mean()) * 100

# 2. Determine if the prediction based on rank was correct
# A "correct" prediction is when the team with the lower rank number wins.
def is_prediction_correct(row):
    # Case 1: Home team has better rank
    if lr.predict_proba([[
        row['Home_Rank'] - row['Away_Rank']
    ]])[0][1] >= 0.5:
        return row['Home_Won'] == 1
    else:
        return row['Home_Won'] == 0

df['Prediction_Correct'] = df.apply(is_prediction_correct, axis=1)

# 3. Calculate the prediction accuracy for each Rank_Age
# This is the percentage of times the prediction was correct.
accuracy_by_age = df.groupby('Rank_Age')['Prediction_Correct'].mean().reset_index()
accuracy_by_age.rename(columns={'Prediction_Correct': 'Prediction_Accuracy'}, inplace=True)

# Convert accuracy to percentage for better readability in the plot
accuracy_by_age['Prediction_Accuracy'] = accuracy_by_age['Prediction_Accuracy'] * 100

# 4. Create the visualization using Plotly
fig = px.line(
    accuracy_by_age,
    x='Rank_Age',
    y='Prediction_Accuracy',
    title='Predictive Power of Team Rank vs. Age of Rank',
    labels={
        'Rank_Age': 'Age of Rank (in weeks)',
        'Prediction_Accuracy': 'Prediction Accuracy (%)'
    },
    markers=True, # Add markers to each data point
    template='plotly_dark'
)

fig.add_hline(
    y=home_win_benchmark, 
    line_dash="dot",  # Style of the line (e.g., 'dot', 'dash')
    annotation_text=f"Home Win % Benchmark ({home_win_benchmark:.1f}%)", 
    annotation_position="bottom right",
    annotation_font_color="yellow"
)

fig.update_layout(
    xaxis_title='Age of Rank (0 = Most Current)',
    yaxis_title='Accuracy of Rank in Predicting Winner (%)',
    yaxis=dict(
        tickformat=".1f",  # Format y-axis labels as percentages
        # range=[0, 100]       # Set y-axis range from 0 to 1
    ),
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()

In [90]:
import pandas as pd
import numpy as np
import plotly.express as px

df = hrank_df.copy()

home_win_benchmark = (df['Home_Won'].mean()) * 100

# 2. Create Rank Difference Buckets
# We calculate the absolute difference in rank to measure how close the matchup is.
df['Rank_Diff'] = abs(df['Home_Rank'] - df['Away_Rank'])

# Define the bins and labels for bucketing the rank differences.
bins = [0, 5, 15, 22, 100] # Bins for 1-5, 6-15, 16-40, 41+
labels = [
    '1-5 (Very Close)', 
    '6-15 (Close)', 
    '16-22 (Mismatch)', 
    '22+ (Large Mismatch)'
]
df['Rank_Diff_Bucket'] = pd.cut(df['Rank_Diff'], bins=bins, labels=labels, right=True)

# Drop any rows where a bucket couldn't be assigned (e.g., if Rank_Diff was 0)
df.dropna(subset=['Rank_Diff_Bucket'], inplace=True)


# 2. Determine if the prediction based on rank was correct
# A "correct" prediction is when the team with the lower rank number wins.
def is_prediction_correct(row):
    # Case 1: Home team has better rank
    if lr.predict_proba([[
        row['Home_Rank'] - row['Away_Rank']
    ]])[0][1] >= 0.5:
        return row['Home_Won'] == 1
    else:
        return row['Home_Won'] == 0

df['Prediction_Correct'] = df.apply(is_prediction_correct, axis=1)

# 4. Calculate prediction accuracy, now grouped by both Age and Bucket
accuracy_by_group = df.groupby(['Rank_Age', 'Rank_Diff_Bucket'])['Prediction_Correct'].mean().reset_index()
accuracy_by_group.rename(columns={'Prediction_Correct': 'Prediction_Accuracy'}, inplace=True)

# Convert accuracy to percentage
accuracy_by_group['Prediction_Accuracy'] *= 100

# 5. Create the multi-line visualization using Plotly
fig = px.line(
    accuracy_by_group,
    x='Rank_Age',
    y='Prediction_Accuracy',
    color='Rank_Diff_Bucket',  # This creates a separate line for each bucket
    title='Predictive Power vs. Rank Age, Segmented by Matchup Closeness',
    labels={
        'Rank_Age': 'Age of Rank (in weeks)',
        'Prediction_Accuracy': 'Prediction Accuracy (%)',
        'Rank_Diff_Bucket': 'Matchup Closeness (Rank Difference)'
    },
    markers=True,
    template='plotly_dark'
)

fig.add_hline(
    y=home_win_benchmark, 
    line_dash="dot",  # Style of the line (e.g., 'dot', 'dash')
    annotation_text=f"Home Win % Benchmark ({home_win_benchmark:.1f}%)", 
    annotation_position="bottom right",
    annotation_font_color="yellow"
)

fig.update_layout(
    xaxis_title='Age of Rank (0 = Most Current)',
    yaxis_title='Accuracy of Rank in Predicting Winner (%)',
    # title_x=0.5,
    legend_title_text='Rank Difference Bucket',
    yaxis=dict(
        tickformat=".1f",  # Format y-axis labels as percentages
        range=[20, 90]       # Set y-axis range from 0 to 1
    ),
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()

/var/folders/rw/wxydsgm576n8npv02vqd4q500000gn/T/ipykernel_78779/1147342222.py:41: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



- it makes sense that the 1-5 is pretty close to the benchmark as the effect of knowing the rank diff disapears once the teams are pretty close in a 50:50 case, but hte home advantange is maintained

In [91]:
import plotly.express as px
import pandas as pd
import numpy as np

# 1. Calculate the rank difference.
# A negative value means the home team has a better (lower) rank.
df = base_df.copy()
df['Rank_Avg'] = (df['Home_Rank'] + df['Away_Rank']) / 2

# 2. Bucketize the rank difference into intervals of size 4.
# We define the edges of the bins from -32 to +32 with a step of 4.
max_abs_diff = df['Rank_Avg'].abs().max()
# Ensure bins cover the full range of data, stepping by 4
bin_edge = int(np.ceil(max_abs_diff / 4) * 4)
bins = np.arange(1, bin_edge + 1, 4)

# Create meaningful labels for each bucket.
labels = [f'{i} to {i + 3}' for i in bins[:-1]]

df['Rank_Avg_Bucket'] = pd.cut(
    df['Rank_Avg'],
    bins=bins,
    labels=labels,
    right=False, # Intervals are [left, right)
    include_lowest=True
)

# 3. Group by the rank difference bucket and calculate home win probability.
# We use observed=False to include all possible buckets in the output, even if empty.
win_prob_by_bucket = (
    df.groupby('Rank_Avg_Bucket', observed=False)['Home_Won']
    .mean()
    .reset_index(name='Home_Win_Probability')
)

# 4. Create the bar plot.
fig = px.bar(
    win_prob_by_bucket,
    x='Rank_Avg_Bucket',
    y='Home_Win_Probability',
    title='Home Win Probability vs. Bucketed Rank Average',
    labels={
        'Rank_Avg_Bucket': 'Rank Average Bucket ((Home Rank - Away Rank)/2)',
        'Home_Win_Probability': 'Home Win Probability'
    },
    template='plotly_dark'
)

# 5. Update the layout for better readability and aesthetics.
fig.update_layout(
    xaxis_title="Rank Average Bucket",
    yaxis_title="Home Win Probability",
    yaxis=dict(
        tickformat=".0%",  # Format y-axis labels as percentages
        range=[0.5, 0.6]       # Set y-axis range from 0 to 1
    ),
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()


In [92]:
import pandas as pd
import numpy as np
import plotly.express as px

df = hrank_df.copy()

# 1. Create Rank Difference Buckets
df['Rank_Diff'] = abs(df['Home_Rank'] - df['Away_Rank'])
bins = [0, 5, 15, 22, 100] # Bins for 1-5, 6-15, 16-40, 41+
labels = [
    '1-5 (Very Close)', 
    '6-15 (Close)', 
    '16-22 (Mismatch)', 
    '22+ (Large Mismatch)'
]
df['Rank_Diff_Bucket'] = pd.cut(df['Rank_Diff'], bins=bins, labels=labels, right=True)
df.dropna(subset=['Rank_Diff_Bucket'], inplace=True)

# 2. Determine if the prediction was correct
def is_prediction_correct(row):
    # Case 1: Home team has better rank
    if lr.predict_proba([[
        row['Home_Rank'] - row['Away_Rank']
    ]])[0][1] >= 0.5:
        return row['Home_Won'] == 1
    else:
        return row['Home_Won'] == 0
df['Prediction_Correct'] = df.apply(is_prediction_correct, axis=1)

# 3. Pivot data for the heatmap
accuracy_grouped = df.groupby(['Rank_Age', 'Rank_Diff_Bucket'])['Prediction_Correct'].mean().reset_index()
accuracy_pivot = accuracy_grouped.pivot(
    index='Rank_Diff_Bucket', 
    columns='Rank_Age', 
    values='Prediction_Correct'
)

# 4. Create the heatmap
fig = px.imshow(
    accuracy_pivot,
    text_auto=".0%", # Format text in cells as percentages
    aspect="auto",
    color_continuous_scale='RdYlGn', # Red-Yellow-Green scale
    title='Heatmap of Prediction Accuracy'
)

fig.update_layout(
    xaxis_title="Age of Rank (in weeks)",
    yaxis_title="Matchup Closeness (Rank Difference)",
    template="plotly_dark",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()

/var/folders/rw/wxydsgm576n8npv02vqd4q500000gn/T/ipykernel_78779/2467115005.py:31: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [93]:
import pandas as pd
import numpy as np
import plotly.express as px


df = hrank_df.copy()

# 1. Create Rank Difference Buckets
df['Rank_Diff'] = abs(df['Home_Rank'] - df['Away_Rank'])
bins = [0, 5, 15, 22, 100] # Bins for 1-5, 6-15, 16-40, 41+
labels = [
    '1-5 (Very Close)', 
    '6-15 (Close)', 
    '16-22 (Mismatch)', 
    '22+ (Large Mismatch)'
]
df['Rank_Diff_Bucket'] = pd.cut(df['Rank_Diff'], bins=bins, labels=labels, right=True)

# 2. Determine who has the better rank
df['Home_Team_Is_Better'] = df['Home_Rank'] < df['Away_Rank']

# 3. Calculate Home Win % grouped by matchup closeness and who is better
win_rate_by_matchup = df.groupby(['Rank_Diff_Bucket', 'Home_Team_Is_Better'])['Home_Won'].mean().reset_index()
win_rate_by_matchup['Home_Won'] *= 100 # Convert to percentage
win_rate_by_matchup['Home_Team_Is_Better'] = win_rate_by_matchup['Home_Team_Is_Better'].map({True: 'Home Team is Higher-Ranked', False: 'Away Team is Higher-Ranked'})


# 4. Create the grouped bar chart
fig = px.bar(
    win_rate_by_matchup,
    x="Rank_Diff_Bucket",
    y="Home_Won",
    color="Home_Team_Is_Better",
    barmode="group", # Key parameter for side-by-side bars
    title="Home Win % by Matchup Closeness and Rank Advantage",
    text_auto=".1f", # Display value on top of bars
    labels={
        "Rank_Diff_Bucket": "Matchup Closeness",
        "Home_Won": "Home Team Win Percentage (%)",
        "Home_Team_Is_Better": "Rank Scenario"
    }
)

fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.update_layout(template="plotly_dark", title_x=0.5)
fig.show()

/var/folders/rw/wxydsgm576n8npv02vqd4q500000gn/T/ipykernel_78779/2498668123.py:23: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [94]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

df = base_df.copy()

# 1. Create bins for the home team's rest days
bins = [0, 4, 6, 8, 13, 100]
labels = ['0-4 Days (Short)', '5-6 Days', '7-8 Days (Standard)', '9-13 Days (Long)', '14+ Days (Bye Week)']
df['Home_Rest_Bucket'] = pd.cut(df['Home_Days_Since_Last_Game'], bins=bins, labels=labels, right=True)

# 2. Calculate home win percentage for each bucket
home_win_by_rest = df.groupby('Home_Rest_Bucket')['Home_Won'].mean().reset_index()
home_win_by_rest['Home_Won'] *= 100

# 3. Create the bar chart
fig1 = px.bar(
    home_win_by_rest,
    x='Home_Rest_Bucket',
    y='Home_Won',
    title='Home Team Win Percentage by Their Days of Rest',
    labels={
        'Home_Rest_Bucket': 'Home Team Days Since Last Game',
        'Home_Won': 'Home Win Percentage (%)'
    },
    text_auto='.1f' # Format text on bars
)

fig1.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig1.update_layout(template="plotly_dark", title_x=0.5)
fig1.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
fig1.show()

/var/folders/rw/wxydsgm576n8npv02vqd4q500000gn/T/ipykernel_78779/2129685109.py:14: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [95]:
df = base_df.copy()

# 1. Calculate the rest day advantage for the home team
df['Rest_Advantage_Home'] = df['Home_Days_Since_Last_Game'] - df['Away_Days_Since_Last_Game']

# 2. Create bins for the rest advantage
adv_bins = [-100, -8, -4, 3, 7, 100]
adv_labels = ['Away Has 8+ Days More', 'Away Has 4-7 Days More', 'Similar Rest (+/- 3 Days)', 'Home Has 4-7 Days More', 'Home Has 8+ Days More']
df['Rest_Advantage_Bucket'] = pd.cut(df['Rest_Advantage_Home'], bins=adv_bins, labels=adv_labels, right=True)

# 3. Calculate home win percentage and add a benchmark line for overall home win %
home_win_by_adv = df.groupby('Rest_Advantage_Bucket')['Home_Won'].mean().reset_index()
home_win_by_adv['Home_Won'] *= 100
overall_home_win_pct = df['Home_Won'].mean() * 100

# 4. Create the bar chart
fig2 = px.bar(
    home_win_by_adv,
    x='Rest_Advantage_Bucket',
    y='Home_Won',
    title='Home Win Percentage Based on Rest Advantage',
    labels={
        'Rest_Advantage_Bucket': 'Rest Advantage (Home Days - Away Days)',
        'Home_Won': 'Home Win Percentage (%)'
    },
    text_auto='.1f'
)

# Add a benchmark line for context
fig2.add_hline(
    y=overall_home_win_pct, 
    line_dash="dot",
    annotation_text=f"Overall Home Win Rate: {overall_home_win_pct:.1f}%",
    annotation_position="bottom right"
)

fig1.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig2.update_layout(template="plotly_dark", title_x=0.5)
fig2.update_traces(texttemplate='%{y:.1f}%', textposition='outside')
fig2.show()

/var/folders/rw/wxydsgm576n8npv02vqd4q500000gn/T/ipykernel_78779/1072091810.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [96]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

df = base_df.copy()

# --- 1. Engineer All Three Feature Types ---

# Method A: Raw Win Percentage
df['Home_Raw_Win_Pct'] = (df['Home_Wins'] / df['Home_Games_Played']).fillna(0.5)
df['Away_Raw_Win_Pct'] = (df['Away_Wins'] / df['Away_Games_Played']).fillna(0.5)
df['Raw_Diff'] = df['Home_Raw_Win_Pct'] - df['Away_Raw_Win_Pct']

# Method B: Additive Smoothing (k=2)
k = 4
df['Home_Smoothed_Pct'] = (df['Home_Wins'] + k) / (df['Home_Games_Played'] + 2*k)
df['Away_Smoothed_Pct'] = (df['Away_Wins'] + k) / (df['Away_Games_Played'] + 2*k)
df['Smoothed_Diff'] = df['Home_Smoothed_Pct'] - df['Away_Smoothed_Pct']

# Method C: Weighted Win Rate (C=8, Rank as Prior)
C = 4
max_rank = max(df['Home_Rank'].max(), df['Away_Rank'].max()) # Find max rank for normalization

# Normalize rank to a 0-1 strength score (higher is better)
df['Home_Prior_Strength'] = 1 - (df['Home_Rank'] - 1) / (max_rank - 1)
df['Away_Prior_Strength'] = 1 - (df['Away_Rank'] - 1) / (max_rank - 1)

df['Home_Confidence_Weight'] = df['Home_Games_Played'] / (df['Home_Games_Played'] + C)
df['Away_Confidence_Weight'] = df['Away_Games_Played'] / (df['Away_Games_Played'] + C)

df['Home_Weighted_Rate'] = (df['Home_Confidence_Weight'] * df['Home_Raw_Win_Pct']) + ((1 - df['Home_Confidence_Weight']) * df['Home_Prior_Strength'])
df['Away_Weighted_Rate'] = (df['Away_Confidence_Weight'] * df['Away_Raw_Win_Pct']) + ((1 - df['Away_Confidence_Weight']) * df['Away_Prior_Strength'])
df['Weighted_Diff'] = df['Home_Weighted_Rate'] - df['Away_Weighted_Rate']

corr_raw = df['Raw_Diff'].corr(df['Home_Won'])
corr_smoothed = df['Smoothed_Diff'].corr(df['Home_Won'])
corr_weighted = df['Weighted_Diff'].corr(df['Home_Won'])

print(f"Pearson Correlation (Raw): {corr_raw:.4f}")
print(f"Pearson Correlation (Smoothed): {corr_smoothed:.4f}")
print(f"Pearson Correlation (Weighted): {corr_weighted:.4f}")

# --- 2. Bin each metric and calculate observed win rate ---

def create_plot_data(df, diff_col, method_name, num_bins=10):
    # Create bins for the difference column
    df['Bin'] = pd.cut(df[diff_col], bins=num_bins)
    
    # Group by bin and calculate the data needed for plotting
    plot_df = df.groupby('Bin', observed=False).agg(
        Observed_Home_Win_Pct=('Home_Won', 'mean'),
        Bin_Midpoint=(diff_col, 'mean'),
        Num_Games=('Home_Won', 'count')
    ).reset_index()
    
    # Filter out bins with very few games as they are noisy
    plot_df = plot_df[plot_df['Num_Games'] > 20].copy()
    
    plot_df['Method'] = method_name
    return plot_df[['Bin_Midpoint', 'Observed_Home_Win_Pct', 'Method']]

# Create new labels for the legend that include the correlation coefficient (ρ)
label_raw = f'Raw Win % Difference (ρ = {corr_raw:.2f})'
label_smoothed = f'Additive Smoothing (k={k}) (ρ = {corr_smoothed:.2f})'
label_weighted = f'Weighted Rate (C={C}) (ρ = {corr_weighted:.2f})'

plot_data_raw = create_plot_data(df, 'Raw_Diff', label_raw)
plot_data_smoothed = create_plot_data(df, 'Smoothed_Diff', label_smoothed)
plot_data_weighted = create_plot_data(df, 'Weighted_Diff', label_weighted)

# Combine into a single DataFrame for Plotly
final_plot_df = pd.concat([plot_data_raw, plot_data_smoothed, plot_data_weighted])
final_plot_df['Observed_Home_Win_Pct'] *= 100 # Convert to percentage

# --- 3. Create the Plotly Visualization ---

fig = px.line(
    final_plot_df,
    x='Bin_Midpoint',
    y='Observed_Home_Win_Pct',
    color='Method',
    markers=True,
    title='Comparing Predictive Power of Feature Engineering Methods'
)

# Add reference lines for better interpretation
fig.add_vline(x=0, line_dash="dash", line_color="grey", annotation_text="Teams are 'Equal'")
fig.add_hline(y=50, line_dash="dot", line_color="grey", annotation_text="50% Win Rate")

fig.update_layout(
    xaxis_title="Calculated Strength Difference (Home - Away)",
    yaxis_title="Observed Home Win Percentage (%)",
    # yaxis_range=[0, 100],
    template="plotly_dark",
    legend_title="Feature Engineering Method",
    title_x=0.5
)

fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()

Pearson Correlation (Raw): 0.2209
Pearson Correlation (Smoothed): 0.2679
Pearson Correlation (Weighted): 0.3175


In [100]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

df = base_df.copy()

# --- 1. Engineer All Three Feature Types ---

# Method A: Raw Win Percentage
df['Home_Raw_Win_Pct'] = (df['Home_Wins'] / df['Home_Games_Played']).fillna(0.5)
df['Away_Raw_Win_Pct'] = (df['Away_Wins'] / df['Away_Games_Played']).fillna(0.5)
df['Raw_Diff'] = df['Home_Raw_Win_Pct'] - df['Away_Raw_Win_Pct']

# Method B: Additive Smoothing (k=2)
k = 2
df['Home_Smoothed_Pct'] = (df['Home_Wins'] + k) / (df['Home_Games_Played'] + 2*k)
df['Away_Smoothed_Pct'] = (df['Away_Wins'] + k) / (df['Away_Games_Played'] + 2*k)
df['Smoothed_Diff'] = df['Home_Smoothed_Pct'] - df['Away_Smoothed_Pct']

# Method C: Weighted Win Rate (C=8, Rank as Prior)
C = 4
max_rank = max(df['Home_Rank'].max(), df['Away_Rank'].max()) # Find max rank for normalization

# Normalize rank to a 0-1 strength score (higher is better)
df['Home_Prior_Strength'] = 1 - (df['Home_Rank'] - 1) / (max_rank - 1)
df['Away_Prior_Strength'] = 1 - (df['Away_Rank'] - 1) / (max_rank - 1)

df['Home_Confidence_Weight'] = df['Home_Games_Played'] / (df['Home_Games_Played'] + C)
df['Away_Confidence_Weight'] = df['Away_Games_Played'] / (df['Away_Games_Played'] + C)

df['Home_Weighted_Rate'] = (df['Home_Confidence_Weight'] * df['Home_Raw_Win_Pct']) + ((1 - df['Home_Confidence_Weight']) * df['Home_Prior_Strength'])
df['Away_Weighted_Rate'] = (df['Away_Confidence_Weight'] * df['Away_Raw_Win_Pct']) + ((1 - df['Away_Confidence_Weight']) * df['Away_Prior_Strength'])
df['Weighted_Diff'] = df['Home_Weighted_Rate'] - df['Away_Weighted_Rate']

# We select games where the home team has played fewer than 6 games.
df = df[df['Home_Games_Played'] < 6].copy()

corr_raw = df['Raw_Diff'].corr(df['Home_Won'])
corr_smoothed = df['Smoothed_Diff'].corr(df['Home_Won'])
corr_weighted = df['Weighted_Diff'].corr(df['Home_Won'])

print(f"Pearson Correlation (Raw): {corr_raw:.4f}")
print(f"Pearson Correlation (Smoothed): {corr_smoothed:.4f}")
print(f"Pearson Correlation (Weighted): {corr_weighted:.4f}")

# --- 2. Bin each metric and calculate observed win rate ---

def create_plot_data(df, diff_col, method_name, num_bins=10):
    # Create bins for the difference column
    df['Bin'] = pd.cut(df[diff_col], bins=num_bins)
    
    # Group by bin and calculate the data needed for plotting
    plot_df = df.groupby('Bin', observed=False).agg(
        Observed_Home_Win_Pct=('Home_Won', 'mean'),
        Bin_Midpoint=(diff_col, 'mean'),
        Num_Games=('Home_Won', 'count')
    ).reset_index()
    
    # Filter out bins with very few games as they are noisy
    plot_df = plot_df[plot_df['Num_Games'] > 20].copy()
    
    plot_df['Method'] = method_name
    return plot_df[['Bin_Midpoint', 'Observed_Home_Win_Pct', 'Method']]

# Create new labels for the legend that include the correlation coefficient (ρ)
label_raw = f'Raw Win % Difference (ρ = {corr_raw:.2f})'
label_smoothed = f'Additive Smoothing (k={k}) (ρ = {corr_smoothed:.2f})'
label_weighted = f'Weighted Rate (C={C}) (ρ = {corr_weighted:.2f})'

plot_data_raw = create_plot_data(df, 'Raw_Diff', label_raw)
plot_data_smoothed = create_plot_data(df, 'Smoothed_Diff', label_smoothed)
plot_data_weighted = create_plot_data(df, 'Weighted_Diff', label_weighted)

# plot_data_raw = create_plot_data(df, 'Raw_Diff', 'Raw Win % Difference')
# plot_data_smoothed = create_plot_data(df, 'Smoothed_Diff', f'Additive Smoothing (k={k})')
# plot_data_weighted = create_plot_data(df, 'Weighted_Diff', f'Weighted Rate (C={C}, Rank Prior)')

# Combine into a single DataFrame for Plotly
final_plot_df = pd.concat([plot_data_raw, plot_data_smoothed, plot_data_weighted])
final_plot_df['Observed_Home_Win_Pct'] *= 100 # Convert to percentage

# --- 3. Create the Plotly Visualization ---

fig = px.line(
    final_plot_df,
    x='Bin_Midpoint',
    y='Observed_Home_Win_Pct',
    color='Method',
    markers=True,
    title='Comparing Predictive Power of Feature Engineering Methods (Only First 6 Games of the Season)'
)

# Add reference lines for better interpretation
fig.add_vline(x=0, line_dash="dash", line_color="grey", annotation_text="Teams are 'Equal'")
fig.add_hline(y=50, line_dash="dot", line_color="grey", annotation_text="50% Win Rate")

fig.update_layout(
    xaxis_title="Calculated Strength Difference (Home - Away)",
    yaxis_title="Observed Home Win Percentage (%)",
    # yaxis_range=[0, 100],
    template="plotly_dark",
    legend_title="Feature Engineering Method",
    title_x=0.5
)

fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    )
)

fig.show()

Pearson Correlation (Raw): 0.1155
Pearson Correlation (Smoothed): 0.1420
Pearson Correlation (Weighted): 0.2832


In [99]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


df = base_df.copy()

# --- 1. Engineer the Weighted Win Rate feature (for comparison) ---
df['Home_Raw_Win_Pct'] = (df['Home_Wins'] / df['Home_Games_Played']).fillna(0.5)
df['Away_Raw_Win_Pct'] = (df['Away_Wins'] / df['Away_Games_Played']).fillna(0.5)
C = 4
max_rank = max(df['Home_Rank'].max(), df['Away_Rank'].max())
df['Home_Prior_Strength'] = 1 - (df['Home_Rank'] - 1) / (max_rank - 1)
df['Away_Prior_Strength'] = 1 - (df['Away_Rank'] - 1) / (max_rank - 1)
df['Home_Confidence_Weight'] = df['Home_Games_Played'] / (df['Home_Games_Played'] + C)
df['Away_Confidence_Weight'] = df['Away_Games_Played'] / (df['Away_Games_Played'] + C)
df['Home_Weighted_Rate'] = (df['Home_Confidence_Weight'] * df['Home_Raw_Win_Pct']) + ((1 - df['Home_Confidence_Weight']) * df['Home_Prior_Strength'])
df['Away_Weighted_Rate'] = (df['Away_Confidence_Weight'] * df['Away_Raw_Win_Pct']) + ((1 - df['Away_Confidence_Weight']) * df['Away_Prior_Strength'])
df['Weighted_Diff'] = df['Home_Weighted_Rate'] - df['Away_Weighted_Rate']

# --- 3. Calculate Pearson Correlation for each predictor ---
# Note: We expect spread to have a NEGATIVE correlation with Home_Won.
corr_spread = df['Spread'].corr(df['Home_Won'])
corr_weighted = df['Weighted_Diff'].corr(df['Home_Won'])

print(f"Pearson Correlation (Spread): {corr_spread:.4f}")
print(f"Pearson Correlation (Weighted Diff): {corr_weighted:.4f}")

# --- 4. Bin metrics and create plot data for both predictors ---
def create_plot_data(df, col_name, method_name, num_bins=10):
    df['Bin'] = pd.cut(df[col_name], bins=num_bins, labels=False, duplicates='drop')
    plot_df = df.groupby('Bin').agg(
        Observed_Home_Win_Pct=('Home_Won', 'mean'),
        Bin_Midpoint=(col_name, 'mean'),
        Num_Games=('Home_Won', 'count')
    ).reset_index()
    plot_df = plot_df[plot_df['Num_Games'] > 10].copy()
    plot_df['Method'] = method_name
    return plot_df[['Bin_Midpoint', 'Observed_Home_Win_Pct', 'Method']]

label_spread = f'Betting Spread (ρ = {corr_spread:.2f})'
label_weighted = f'Weighted Win Rate Model (ρ = {corr_weighted:.2f})'

plot_data_spread = create_plot_data(df, 'Spread', label_spread)
plot_data_weighted = create_plot_data(df, 'Weighted_Diff', label_weighted)

final_plot_df = plot_data_spread # pd.concat([plot_data_spread, plot_data_weighted])
final_plot_df['Observed_Home_Win_Pct'] *= 100

# --- 5. Create the Plotly Visualization ---
fig = px.line(
    final_plot_df,
    x='Bin_Midpoint',
    y='Observed_Home_Win_Pct',
    color='Method',
    markers=True,
    title='Spread Predictive Power'
)

fig.add_hline(y=50, line_dash="dot", line_color="grey", annotation_text="50% Win Rate")
# A vertical line at x=0 is less meaningful here as the scales are different, so we omit it.

fig.update_layout(
    xaxis_title="Spread",
    yaxis_title="Observed Home Win Percentage (%)",
    yaxis_range=[0, 100],
    template="plotly_dark",
    legend_title="Predictive Method",
    title_x=0.5
)

fig.show()

Pearson Correlation (Spread): -0.3811
Pearson Correlation (Weighted Diff): 0.3175


###  Model

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

In [34]:
db = duckdb.connect(':memory:')
Xy = db.sql(
"""
SELECT
    *
FROM hrank_df
-- WHERE Year = 2024 AND (Home_Team = 'Kansas City' OR Away_Team = 'Kansas City')
QUALIFY
    ROW_NUMBER() OVER (PARTITION BY Year, Week, Home_Team, Away_Team ORDER BY MD5(CONCAT(Year, Week, Home_Team, Away_Team, Rank_Age))) = 1
ORDER BY Year, Week
"""
).df()
db.close()

Xy.head()


,Year,Week,Date,Home_Team,Away_Team,Is_Neutral,Home_Days_Since_Last_Game,Away_Days_Since_Last_Game,Home_Games_Played,Away_Games_Played,Home_Wins,Away_Wins,Home_Losses,Away_Losses,Rank_Age,Home_Rank,Away_Rank,Home_Won
0,2013,1,2013-09-08,Carolina,Seattle,0,14,14,0,0,0.0,0.0,0.0,0.0,0,29,6,0
1,2013,1,2013-09-09,Washington,Philadelphia,0,14,14,0,0,0.0,0.0,0.0,0.0,0,26,14,0
2,2013,1,2013-09-08,Buffalo,New England,0,14,14,0,0,0.0,0.0,0.0,0.0,0,22,1,0
3,2013,1,2013-09-09,San Diego,Houston,0,14,14,0,0,0.0,0.0,0.0,0.0,0,17,13,0
4,2013,1,2013-09-08,NY Jets,Tampa Bay,0,14,14,0,0,0.0,0.0,0.0,0.0,0,21,19,1


In [21]:
X = Xy.drop(columns=['Date', 'Year', 'Home_Team', 'Away_Team', 'Home_Won'])
y = Xy['Home_Won']

In [22]:
def additive_smoothing_win_rate_diff(X, k=2):
    htw = (X["Home_Wins"] + k) / (X["Home_Games_Played"] + 2*k)
    atw = (X["Away_Wins"] + k) / (X["Away_Games_Played"] + 2*k)
    return (htw - atw).values.reshape(-1, 1)

def weighted_win_rate_diff(X, C=4, max_rank=32):
    X_ = X.copy()
    X_["Home_Raw_Win_Pct"] = (X_["Home_Wins"] / X_["Home_Games_Played"]).fillna(0.5)    
    X_["Away_Raw_Win_Pct"] = (X_["Away_Wins"] / X_["Away_Games_Played"]).fillna(0.5)

    hps = 1 - (X_["Home_Rank"] - 1) / (max_rank - 1)
    aps = 1 - (X_["Away_Rank"] - 1) / (max_rank - 1)
    
    hcw = X_["Home_Games_Played"] / (X_["Home_Games_Played"] + C)
    acw = X_["Away_Games_Played"] / (X_["Away_Games_Played"] + C)
    
    hwr = (hcw * X_["Home_Raw_Win_Pct"]) + ((1 - hcw) * hps)
    awr = (acw * X_["Away_Raw_Win_Pct"]) + ((1 - acw) * aps)
    
    return (hwr - awr).values.reshape(-1, 1)

In [23]:
def create_model(
    penalty='l2',
    C=1.0,
    tol=1e-4,
    solver='lbfgs',
    max_iter=1000,
    wr_C=2,
    random_state=42,
    **kwargs
):

    # Define the column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('passthrough', 'passthrough', ["Is_Neutral", "Rank_Age"]),
            # ('Diff_Rank', 
            #     FunctionTransformer(
            #         lambda X_: (X_['Home_Rank'] - X_['Away_Rank']).values.reshape(-1, 1)
            #     ), 
            #     ['Home_Rank', 'Away_Rank']
            # ),
            ('Diff_Days_Rest', 
                FunctionTransformer(
                    lambda X_: (X_['Home_Days_Since_Last_Game'] - X_['Away_Days_Since_Last_Game']).values.reshape(-1, 1)
                ), 
                ['Home_Days_Since_Last_Game', 'Away_Days_Since_Last_Game']
            ),
            ('Win_Rate_Diff', 
                FunctionTransformer(lambda X_: weighted_win_rate_diff(X_, C=wr_C)), 
                ['Home_Wins', 'Home_Games_Played', 'Away_Wins', 'Away_Games_Played', 'Home_Rank', 'Away_Rank']
            ),
            ('Season_Stage', 
                Pipeline([
                    ("transform", FunctionTransformer(
                        lambda X_: X_["Week"].map(lambda x: 0 if x <= 6 else 1 if x <= 12 else 2).values.reshape(-1, 1)
                        )
                    ),
                    ("encode", OneHotEncoder())
                ]), 
                ['Week']
            ),
        ]
    )

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('clf', LogisticRegression(
            penalty=penalty,
            C=C, tol=tol,
            solver=solver,
            max_iter=max_iter,
            random_state=random_state,
            **kwargs
        ))
    ])
    return pipeline

In [24]:
# model = create_model()
# pd.DataFrame(
#     model.named_steps['preprocessor'].fit_transform(X, y),
#     columns=["Is_Neutral", "Rank_Age", "Diff_Rank", "Diff_Days_Rest", "Win_Rate_Diff", "1", "2", "3"]
# )

In [25]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    pipeline = create_model(
        C=trial.suggest_float('C', 0.0001, 10.0, log=True),
        tol=trial.suggest_float('tol', 1e-6, 1e-2, log=True),
        wr_C=trial.suggest_int("wr_C", 1, 18, step=1)
    )

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_log_loss')
    return -scores.mean()

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250, show_progress_bar=True)

print("Best trial:")
print(study.best_trial)

  0%|          | 0/250 [00:00<?, ?it/s]

Best trial:
FrozenTrial(number=230, state=1, values=[0.6337264694305178], datetime_start=datetime.datetime(2025, 7, 13, 14, 3, 24, 786864), datetime_complete=datetime.datetime(2025, 7, 13, 14, 3, 24, 846991), params={'C': 0.4866299518840896, 'tol': 0.0004716016033283257, 'wr_C': 9}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'C': FloatDistribution(high=10.0, log=True, low=0.0001, step=None), 'tol': FloatDistribution(high=0.01, log=True, low=1e-06, step=None), 'wr_C': IntDistribution(high=18, log=False, low=1, step=1)}, trial_id=230, value=None)


0.6337660326316931
0.6340747975804597

In [26]:
# Get the best parameters from the Optuna study
best_params = study.best_trial.params

# Create the model using the optimal parameters
model = create_model(**best_params)

# Train the model on all data
model.fit(X, y)

,steps,"[('preprocessor', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('passthrough', ...), ('Diff_Days_Rest', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [27]:
X_test = Xy.copy().drop(columns=["Home_Team", "Away_Team", "Year"]) # [ (Xy.Year == 2024) & (Xy.Week == 1) ]
X_test["p"] = model.predict_proba(X_test.drop(columns=["Home_Won"]))[:, 1]
X_test["Home_Won-p"] = X_test["p"].map(lambda p: 1 if p >= 0.5 else 0)
X_test["Home_Won-home"] = 1
X_test

,Week,Date,Is_Neutral,Home_Days_Since_Last_Game,Away_Days_Since_Last_Game,Home_Games_Played,Away_Games_Played,Home_Wins,Away_Wins,Home_Losses,Away_Losses,Rank_Age,Home_Rank,Away_Rank,Home_Won,p,Home_Won-p,Home_Won-home
0,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,18,15,1,0.481991,0,1
1,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,7,11,1,0.594335,1,1
2,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,4,3,1,0.514407,1,1
3,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,27,24,1,0.481991,0,1
4,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,8,23,1,0.749382,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,18,2025-01-05,0,8,11,16,16,9.0,15.0,7.0,1.0,16,30,2,1,0.292993,0,1
3131,18,2025-01-04,0,10,7,16,16,10.0,8.0,6.0,8.0,16,13,11,0,0.619206,1,1
3132,18,2025-01-05,0,7,10,16,16,11.0,4.0,5.0,12.0,8,6,15,0,0.722885,1,1
3133,18,2025-01-05,0,7,7,16,16,13.0,3.0,3.0,13.0,8,7,28,1,0.823259,1,1


In [28]:
(X_test["Home_Won"] == X_test["Home_Won-home"]).sum() / len(X_test)

np.float64(0.5505582137161085)

In [29]:
(X_test["Home_Won"] == X_test["Home_Won-p"]).sum() / len(X_test)

np.float64(0.6440191387559808)

### Save models

In [30]:
import cloudpickle as pickle

with open('./models/lr_no_spread.pkl', 'wb') as m:
    pickle.dump(model, m)

### Viz Model

In [33]:
with open('./models/lr_no_spread.pkl', 'rb') as f:
    model = pickle.load(f)

In [41]:
X_test = Xy.copy().drop(columns=["Home_Team", "Away_Team", "Year"])
X_test["p"] = model.predict_proba(X_test.drop(columns=["Home_Won"]))[:, 1]
X_test["p"] = np.abs(X_test["p"] - 0.5)  # Convert to distance from 50%
X_test

,Week,Date,Is_Neutral,Home_Days_Since_Last_Game,Away_Days_Since_Last_Game,Home_Games_Played,Away_Games_Played,Home_Wins,Away_Wins,Home_Losses,Away_Losses,Rank_Age,Home_Rank,Away_Rank,Home_Won,p
0,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,29,6,0,0.297244
1,1,2013-09-09,0,14,14,0,0,0.0,0.0,0.0,0.0,0,26,14,0,0.158307
2,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,22,1,0,0.275468
3,1,2013-09-09,0,14,14,0,0,0.0,0.0,0.0,0.0,0,17,13,0,0.034177
4,1,2013-09-08,0,14,14,0,0,0.0,0.0,0.0,0.0,0,21,19,1,0.001803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,18,2025-01-05,0,8,10,16,16,10.0,9.0,6.0,7.0,11,22,23,0,0.082992
3131,18,2025-01-05,0,7,8,16,16,4.0,10.0,12.0,6.0,17,27,18,0,0.096209
3132,18,2025-01-05,0,7,7,16,16,8.0,4.0,8.0,12.0,4,23,32,0,0.177233
3133,18,2025-01-05,0,6,7,16,16,14.0,14.0,2.0,2.0,17,4,23,1,0.178429


In [42]:
import pandas as pd
import plotly.graph_objects as go

# Group by Rank_Age and calculate mean and std of predicted probability
agg = X_test.groupby("Rank_Age")["p"].agg(["mean", "std"]).reset_index()

fig = go.Figure()

# Add mean line
fig.add_trace(go.Scatter(
    x=agg["Rank_Age"],
    y=agg["mean"],
    mode="lines+markers",
    name="Mean Probability",
    line=dict(color="deepskyblue", width=3),
    marker=dict(size=7)
))

# Add shaded area for mean ± std
fig.add_trace(go.Scatter(
    x=pd.concat([agg["Rank_Age"], agg["Rank_Age"][::-1]]),
    y=pd.concat([agg["mean"] + agg["std"], (agg["mean"] - agg["std"])[::-1]]),
    fill="toself",
    fillcolor="rgba(30,144,255,0.2)",
    line=dict(color="rgba(255,255,255,0)"),
    hoverinfo="skip",
    showlegend=True,
    name="±1 Std Dev"
))

fig.update_layout(
    title="Predicted Probability by Rank_Age (Mean ± Std Dev)",
    xaxis_title="Rank_Age",
    yaxis_title="Predicted Probability (p)",
    template="plotly_dark",
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="white"
    ),
    legend_title_text="Legend"
)

fig.show()